# MuSoHu Dataset Interfacing

In this notebook we build a script to extract relevant data from the MuSoHu dataset and prepare it for downstream ingestion, synthetic data generation, training and instruction tuning.

Installing dependencies

In [119]:
!pip install omegaconf bagpy tqdm
!pip install PyntCloud

Ingesting pickles generated by the bag file parser

In [192]:
%load_ext autoreload
%autoreload 2
from dataloader import SocialNavDataset

dataset = SocialNavDataset(root='./data/train/samples.pkl')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [193]:
dataset.data

{'past_positions': [array([[ 0.95763622, -0.19005594],
         [ 1.13532803, -0.17450544],
         [ 1.33603153, -0.15185802],
         [ 1.56504049, -0.11961865],
         [ 1.81913723, -0.08049139],
         [ 2.10128839, -0.0345423 ],
         [ 2.40127664,  0.01980859],
         [ 2.72485713,  0.08153264],
         [ 3.06664128,  0.14652076],
         [ 3.42372672,  0.21460577],
         [ 3.79323381,  0.28637545],
         [ 4.17383326,  0.35971529],
         [ 4.56093676,  0.43049252],
         [ 4.95087143,  0.49925058],
         [ 5.3387628 ,  0.56624847],
         [ 5.72056844,  0.62947523]]),
  array([[ 1.13532803, -0.17450544],
         [ 1.33603153, -0.15185802],
         [ 1.56504049, -0.11961865],
         [ 1.81913723, -0.08049139],
         [ 2.10128839, -0.0345423 ],
         [ 2.40127664,  0.01980859],
         [ 2.72485713,  0.08153264],
         [ 3.06664128,  0.14652076],
         [ 3.42372672,  0.21460577],
         [ 3.79323381,  0.28637545],
         [ 4.17383

In [194]:
dataset = SocialNavDataset(root='./data/train/samples.pkl')


In [201]:
sample_0 = dataset.__getitem__(6)
sample_0

{'past_positions': tensor([[2.4013, 0.0198],
         [2.7249, 0.0815],
         [3.0666, 0.1465],
         [3.4237, 0.2146],
         [3.7932, 0.2864],
         [4.1738, 0.3597],
         [4.5609, 0.4305],
         [4.9509, 0.4993],
         [5.3388, 0.5662],
         [5.7206, 0.6295],
         [6.0902, 0.6863],
         [6.4565, 0.7358],
         [6.8354, 0.7753],
         [7.2219, 0.8048],
         [7.6083, 0.8261],
         [7.9894, 0.8388]]),
 'future_positions': tensor([[ 0.3718, -0.0033],
         [ 0.7336, -0.0157],
         [ 1.0777, -0.0328],
         [ 1.4039, -0.0541],
         [ 1.7159, -0.0796],
         [ 2.0054, -0.1077],
         [ 2.2714, -0.1346],
         [ 2.5156, -0.1619]]),
 'past_yaw': tensor([0.1669, 0.1634, 0.1599, 0.1557, 0.1490, 0.1363, 0.1237, 0.1097, 0.0905,
         0.0695, 0.0519, 0.0383, 0.0285, 0.0233, 0.0211, 0.0235]),
 'future_yaw': tensor([0.0312, 0.0442, 0.0602, 0.0787, 0.1018, 0.1277, 0.1547, 0.1844]),
 'past_vw': tensor([1.1540, 0.1269, 1.1731, 0

In [100]:
# Pickle language_modality_frontal_4.txt
import pickle

def read_data(file_path):
    # Initialize dictionary and temporary storage for images and instructions
    data = {}
    images = []
    instructions = []
    dict_count = 1
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip().replace('/t', '')  # Remove tabs and extra spaces
            if line.endswith('.jpg'):  # Check if line is an image path
                images.append(line.replace('\\\\', '\\'))  # Replace / with \ in path
                #print(line)
            else:
                if line:  # Make sure instruction is not empty
                    instructions.append(line)
            
            # Check if we have collected 10 images or reached end of file
            if len(images) == 10:
                data[f'dict{dict_count}'] = {'images': images, 'instructions': instructions}
                images = []  # Reset images list for the next batch
                instructions = []  # Reset instructions list for the next batch
                dict_count += 1

    # Handle any remaining data if less than 10 images
                
    if images:
        data[f'dict{dict_count}'] = {'images': images, 'instructions': instructions}
    
    return data

# Replace 'your_file_path.txt' with the path to your text file
file_path = 'language_modality_frontal_4.txt'
organized_data = read_data(file_path)
print(organized_data)

# Save the organized data to a .pkl file
pkl_path = 'language_modality_frontal_4.pkl'
with open(pkl_path, 'wb') as handle:
    pickle.dump(organized_data, handle)

print(f"Data has been saved to {pkl_path}")


{'dict1': {'images': ['data\\parsed\\ic\\GT\\Spring 2024\\Deep Learning for Robotics\\Project\\raw\\02172023_frontal_approach_4_0\\rgb\\0.jpg', 'data\\parsed\\ic\\GT\\Spring 2024\\Deep Learning for Robotics\\Project\\raw\\02172023_frontal_approach_4_0\\rgb\\10.jpg', 'data\\parsed\\ic\\GT\\Spring 2024\\Deep Learning for Robotics\\Project\\raw\\02172023_frontal_approach_4_0\\rgb\\20.jpg', 'data\\parsed\\ic\\GT\\Spring 2024\\Deep Learning for Robotics\\Project\\raw\\02172023_frontal_approach_4_0\\rgb\\30.jpg'], 'instructions': ["Move forward and you'll see a sitting area to your right with red chairs.", "Go straight, and there's a staircase on your right if you need to go to the upper floors.", 'Keep going straight, you will pass a staircase on your right, leading up.', "Walk down the corridor, you'll see a meeting area to your right with tables and chairs."]}}
Data has been saved to language_modality_frontal_4.pkl


In [110]:
# Unpickle compiled language modality
import pickle
language_dict = []
with open('language_modality_frontal_4.pkl', 'rb') as f:
    language_dict = list(pickle.load(f).values())

# Print the contents of the unpickled data
print(language_dict)

[{'images': ['data\\parsed\\ic\\GT\\Spring 2024\\Deep Learning for Robotics\\Project\\raw\\02172023_frontal_approach_4_0\\rgb\\0.jpg', 'data\\parsed\\ic\\GT\\Spring 2024\\Deep Learning for Robotics\\Project\\raw\\02172023_frontal_approach_4_0\\rgb\\10.jpg', 'data\\parsed\\ic\\GT\\Spring 2024\\Deep Learning for Robotics\\Project\\raw\\02172023_frontal_approach_4_0\\rgb\\20.jpg', 'data\\parsed\\ic\\GT\\Spring 2024\\Deep Learning for Robotics\\Project\\raw\\02172023_frontal_approach_4_0\\rgb\\30.jpg'], 'instructions': ["Move forward and you'll see a sitting area to your right with red chairs.", "Go straight, and there's a staircase on your right if you need to go to the upper floors.", 'Keep going straight, you will pass a staircase on your right, leading up.', "Walk down the corridor, you'll see a meeting area to your right with tables and chairs."]}]


In [116]:
import os
# Code to find an instruction given an image
target_frame = 'data\parsed\\ic\\GT\\Spring 2024\\Deep Learning for Robotics\\Project\\raw\\02172023_frontal_approach_4_0\\rgb\\0.jpg'
target_frame = os.path.normpath(target_frame)

# Extract the image number from the path. Easier than comparing 2 path strings
# Split the path by '\\' to get the individual components
path_components = target_frame.split('\\')

# Get the filename component
filename = path_components[-1]

# Split the filename by '.' to separate the filename and extension
filename_without_extension = filename.split('.')[0]

# Extract the digits from the filename (For this test case should be 0 since it's 0.jpg)
target_digits = ''.join(filter(str.isdigit, filename_without_extension))

print("TARGET DIGITS: ",target_digits)

for dict in language_dict:
    images = dict.get('images', [])
    instructions = dict.get('instructions', [])
    for image, instruction in zip(images, instructions):
        image = os.path.normpath(image)
        path_components = image.split('\\')

        # Get the filename component
        filename = path_components[-1]

        # Split the filename by '.' to separate the filename and extension
        filename_without_extension = filename.split('.')[0]

        # Extract the digits from the filename
        current_digits = ''.join(filter(str.isdigit, filename_without_extension))

        print("CURRENT DIGITS: ", current_digits)

        if current_digits == target_digits:
            print("IMAGE MATCHES TARGET")
            print(f"Instruction for {target_digits}.jpg: ", instruction)

TARGET DIGITS:  0
CURRENT DIGITS:  0
IMAGE MATCHES TARGET
Instruction for 0.jpg:  Move forward and you'll see a sitting area to your right with red chairs.
CURRENT DIGITS:  10
CURRENT DIGITS:  20
CURRENT DIGITS:  30


In [118]:
# After running parser_modified, see instructions for each observation window added. Instructions overlap due to the nature of the overlapping frames in each sample.
import pickle

with open('data/train/samples.pkl', 'rb') as f:
    data = pickle.load(f)

# Print the contents of the unpickled data
print(data)

{'past_positions': [array([[ 0.95763622, -0.19005594],
       [ 1.13532803, -0.17450544],
       [ 1.33603153, -0.15185802],
       [ 1.56504049, -0.11961865],
       [ 1.81913723, -0.08049139],
       [ 2.10128839, -0.0345423 ],
       [ 2.40127664,  0.01980859],
       [ 2.72485713,  0.08153264],
       [ 3.06664128,  0.14652076],
       [ 3.42372672,  0.21460577],
       [ 3.79323381,  0.28637545],
       [ 4.17383326,  0.35971529],
       [ 4.56093676,  0.43049252],
       [ 4.95087143,  0.49925058],
       [ 5.3387628 ,  0.56624847],
       [ 5.72056844,  0.62947523]]), array([[ 1.13532803, -0.17450544],
       [ 1.33603153, -0.15185802],
       [ 1.56504049, -0.11961865],
       [ 1.81913723, -0.08049139],
       [ 2.10128839, -0.0345423 ],
       [ 2.40127664,  0.01980859],
       [ 2.72485713,  0.08153264],
       [ 3.06664128,  0.14652076],
       [ 3.42372672,  0.21460577],
       [ 3.79323381,  0.28637545],
       [ 4.17383326,  0.35971529],
       [ 4.56093676,  0.43049252]